In [1]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Dropout,MaxPooling2D,Flatten
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from PIL import Image
import numpy as np
import os
import pathlib
import cv2

In [2]:
#define a preprocessing function
def pre_func(image):
    image = np.array(image)
    converted_img = (image - 128.0) / 128
    return converted_img

#load images and labels
img_gen = ImageDataGenerator(rotation_range=30,
                             width_shift_range=0.3,
                             height_shift_range=0.3,
                             brightness_range=[0.2,1.0],
                             validation_split=0.2,
                             preprocessing_function=pre_func,
                             dtype='float32')

train_image_gen = img_gen.flow_from_directory("data",target_size=(64,64),color_mode='grayscale',batch_size=1)

Found 98 images belonging to 3 classes.


In [3]:
input_shape=(64,64,1)

model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),   # 1st layer
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(32, (3, 3), activation='relu'),                                        # 2nd layer
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),                                        # 3rd layer
    MaxPooling2D(pool_size=(2, 2)),
#    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
#    Dropout(0.5),
    Dense(3, activation='softmax')
])

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 62, 62, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 29, 29, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 12, 12, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 2304)              0

In [5]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [6]:
print(train_image_gen.class_indices)

{'Closed': 0, 'Opened': 1, 'Un_recognized': 2}


In [7]:
results = model.fit_generator(train_image_gen,epochs=150,steps_per_epoch=98)

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/150
98/98 [==============================] - 2s 19ms/step - loss: 1.1134 - accuracy: 0.3367
Epoch 2/150
98/98 [==============================] - 2s 19ms/step - loss: 1.1141 - accuracy: 0.3469
Epoch 3/150
98/98 [==============================] - 2s 19ms/step - loss: 1.0880 - accuracy: 0.3265
Epoch 4/150
98/98 [==============================] - 2s 19ms/step - loss: 1.1171 - accuracy: 0.4082
Epoch 5/150
98/98 [==============================] - 2s 19ms/step - loss: 1.0972 - accuracy: 0.3980
Epoch 6/150
98/98 [==============================] - 2s 19ms/step - loss: 1.1189 - accuracy: 0.3367
Epoch 7/150
98/98 [==============================] - 2s 19ms/step - loss: 1.0989 - accuracy: 0.4184
Epoch 8/150
98/98 [==============================] - 2s 18ms/step - loss: 1.0809 - accuracy: 0.3878
Epoch 9/150
98/98 [==============================] - 2s 18ms/step - loss: 0.9750 - accuracy: 0.5204
Epoch 10/150
98/98 [====

98/98 [==============================] - 2s 20ms/step - loss: 0.3592 - accuracy: 0.8878
Epoch 79/150
98/98 [==============================] - 2s 19ms/step - loss: 0.2896 - accuracy: 0.9082
Epoch 80/150
98/98 [==============================] - 2s 19ms/step - loss: 0.4945 - accuracy: 0.8265
Epoch 81/150
98/98 [==============================] - 2s 19ms/step - loss: 0.4337 - accuracy: 0.8061
Epoch 82/150
98/98 [==============================] - 2s 19ms/step - loss: 0.3341 - accuracy: 0.8673
Epoch 83/150
98/98 [==============================] - 2s 19ms/step - loss: 0.3608 - accuracy: 0.8673
Epoch 84/150
98/98 [==============================] - 2s 19ms/step - loss: 0.3211 - accuracy: 0.8673
Epoch 85/150
98/98 [==============================] - 2s 19ms/step - loss: 0.3437 - accuracy: 0.8367
Epoch 86/150
98/98 [==============================] - 2s 20ms/step - loss: 0.2016 - accuracy: 0.9286
Epoch 87/150
98/98 [==============================] - 2s 20ms/step - loss: 0.2535 - accuracy: 0.8673
Epo

In [8]:
model.save("model_test.h5")

In [9]:
model.load_weights('model_test.h5')

In [10]:
#preprocessing the test model
test_gen=ImageDataGenerator(dtype='float32')
test_image_gen=test_gen.flow_from_directory("data",target_size=(64,64),color_mode='grayscale',batch_size=1)

images_array = test_image_gen[0][0]
labels_array = test_image_gen[0][1]
for i in range(1,98):
    images_array = np.append(images_array,test_image_gen[i][0])
    labels_array = np.append(labels_array,test_image_gen[i][1])

images_array = images_array.reshape(98,64,64,1)
images_array = images_array.astype('float32')
images_array = (images_array - 128.0) / 128

labels_done = []
labels_array = labels_array.reshape(98,3)
for i in range(len(labels_array)):
    for j in range(0,3):
        if(labels_array[i][j]):
            labels_done = np.append(labels_done,j)

test_labels = labels_done
labels_done = to_categorical(labels_done, 3, dtype = 'float32')

#convert model into TFLM format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

#preprocess representative images
preprocessed_test_images = images_array
preprocessed_test_images = tf.cast(preprocessed_test_images, tf.float32)
tflite_ds = tf.data.Dataset.from_tensor_slices((preprocessed_test_images)).batch(1) #construct a dataset 

def representative_data_gen():
    for input_value in tflite_ds.take(20):
        yield [input_value]
    
converter.representative_dataset = representative_data_gen

#convert model
converted_model = converter.convert()

Found 98 images belonging to 3 classes.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: C:\Users\Japoka\AppData\Local\Temp\tmpc8q13y1z\assets


In [11]:
import pathlib
generated_dir = pathlib.Path("generated/")
generated_dir.mkdir(exist_ok=True, parents=True)
converted_model_file = generated_dir/"test_model_int8.tflite"
converted_model_file.write_bytes(converted_model)

333312

In [12]:
interpreter = tf.lite.Interpreter(model_path=str(converted_model_file))
interpreter.allocate_tensors()

In [13]:
max_samples = 80
input_scale, input_zero_point = interpreter.get_input_details()[0]['quantization']
print(input_scale,input_zero_point)
output_scale, output_zero_point = interpreter.get_output_details()[0]['quantization']
print(output_scale,output_zero_point)

0.0078125 0
0.00390625 -128


In [14]:
# A helper function to evaluate the TF Lite model using "test" dataset.
def evaluate_model(interpreter):
    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]
    
    scale, zero_point = interpreter.get_input_details()[0]['quantization']

    prediction_values = []
    output_buffer = []
    #only consider the result having strong confidence(?%) to reduce lossing accuracy
    higher_threshold = 0
    lowwer_threshold = 0
    
    for test_image in preprocessed_test_images[:max_samples]:
        # Pre-processing: add batch dimension, quantize and convert inputs to int8 to match with
        # the model's input data format.
        test_image = np.expand_dims(test_image, axis=0) #.astype(np.float32)
        test_image = np.int8(test_image / scale + zero_point)
        interpreter.set_tensor(input_index, test_image)

        interpreter.invoke()

        # Find the answer with highest probability
        output = interpreter.tensor(output_index)
        output_buffer = np.append(output_buffer,output()[0][1])
        result = np.argmax(output()[0])
        prediction_values.append(result)
        
    accurate_count_k = 0
    num_data_used = 0
    
    for index in range(len(prediction_values)):
        if(output_buffer[index]>higher_threshold or output_buffer[index]<lowwer_threshold):
            num_data_used +=1
            if prediction_values[index] == test_labels[index]:
                accurate_count_k +=1
                
    #print(num_data_used)    

    accuracy = accurate_count_k * 1.0 / num_data_used
    
    return accuracy * 100

In [15]:
print(str(evaluate_model(interpreter)) + "%")

98.75%
